
# OpenRouter x Python 


Objectif : appeler **OpenRouter** depuis Python en lisant la clé **depuis `.env`** (variable `OPENROUTER_API_KEY`).  
Nous montrons :
1. Chargement des variables d'environnement
2. Appel **non-streaming** (simple)
3. Appel **streaming** (ligne par ligne)

> Prérequis : créer un fichier `.env` à la racine de votre projet contenant :  
> `OPENROUTER_API_KEY=sk-...`


## 0) Dépendances

In [ ]:

# Décommentez si besoin
# !pip install requests python-dotenv -q


## 1) Charger la clé API depuis `.env`

In [1]:

import os
from dotenv import load_dotenv

# Charge .env
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
assert OPENROUTER_API_KEY, "La variable d'environnement OPENROUTER_API_KEY est absente. Ajoutez-la dans votre .env"
print("Clé détectée (tronquée) :", OPENROUTER_API_KEY[:8] + "..." if OPENROUTER_API_KEY else "Aucune")


Clé détectée (tronquée) : sk-or-v1...


## 2) Paramètres simples

In [2]:

import requests

OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"

# Choisissez un modèle auquel votre compte a accès.
# Exemples fréquents (selon disponibilités) :
# - "meta-llama/llama-3.1-70b-instruct"
# - "anthropic/claude-3.5-sonnet"
# - "openai/gpt-4o-mini"
# - "mistralai/mixtral-8x7b-instruct"
MODEL = "meta-llama/llama-3.1-70b-instruct"  # <— changez si nécessaire

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    # Optionnel mais recommandé par OpenRouter (aide à l'attribution & au support)
    "HTTP-Referer": "https://your-site-or-github.com",
    "X-Title": "OpenRouter Minimal Notebook",
}
print("Modèle :", MODEL)


Modèle : meta-llama/llama-3.1-70b-instruct


## 3) Appel 


In [3]:

payload = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": "Tu es un assistant concis et pédagogique."},
        {"role": "user", "content": "Explique en 2 phrases ce qu'est un LLM et donne un cas d'usage concret."}
    ],
    "stream": False  # on veut une seule réponse JSON
}

r = requests.post(OPENROUTER_URL, headers=headers, json=payload)
print("Status:", r.status_code)
print("Content-Type:", r.headers.get("Content-Type"))

data = r.json()  # ok parce que stream=False => un seul JSON
# La réponse standard suit la structure OpenAI-like:
# data["choices"][0]["message"]["content"]
print("\nRéponse :\n", data["choices"][0]["message"]["content"])


Status: 200
Content-Type: application/json

Réponse :
 Un LLM (Large Language Model) est un modèle d'intelligence artificielle entraîné sur de grandes quantités de données textuelles pour générer du texte cohérent et réaliste, capable de comprendre et de répondre à des questions ou à des requêtes. Un exemple concret d'utilisation d'un LLM est un chatbot de support client, comme celui utilisé par une entreprise pour répondre automatiquement aux questions fréquentes des clients et fournir des informations sur ses produits ou services.


## 4) Appel **streaming** (ligne par ligne)

In [4]:

import json

payload_stream = {
    "model": MODEL,
    "messages": [
        {"role": "system", "content": "Tu es un assistant concis et pédagogique."},
        {"role": "user", "content": "Donne 4 bonnes pratiques pour écrire un prompt efficace."}
    ],
    "stream": True  # renverra un flux d'events JSON (SSE/NDJSON)
}

with requests.post(OPENROUTER_URL, headers=headers, json=payload_stream, stream=True) as resp:
    resp.raise_for_status()
    for line in resp.iter_lines():
        if not line:
            continue
        # OpenRouter envoie des events commençant souvent par 'data: {json}'
        raw = line.decode("utf-8")
        if not raw.startswith("data:"):
            continue
        raw = raw[len("data:"):].strip()
        if raw == "[DONE]":
            break
        try:
            event = json.loads(raw)
            delta = event.get("choices", [{}])[0].get("delta", {}).get("content", "")
            if delta:
                print(delta, end="")
        except json.JSONDecodeError:
            # ignore les lignes non-JSON (keep-alive, etc.)
            pass
print("\n— Fin du streaming —")


Voici 4 bonnes pratiques pour écrire un prompt efficace :

1.  **Soyez précis** : Définissez clairement ce que vous voulez obtenir en réponse. Évitez les questions ou les demandes vagues.

2.  **Soyez concis** : Plus votre prompt sera court et concis, plus il sera facile à comprendre et à traiter.

3.  **Utilisez des mots-clés** : Les mots-clés aident à spécifier le contexte et la direction que vous voulez donner à la réponse.

4.  **Soyez spécifique** : Donnez suffisamment de détails pour que la réponse soit pertinente et utile.
— Fin du streaming —
